In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import BinaryType
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import PCA, StandardScaler
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from PIL import Image
import numpy as np
import io
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.types import ArrayType, FloatType
import cv2

2024-05-29 09:56:48.253321: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
if SparkSession.builder.getOrCreate().sparkContext:
    SparkSession.builder.getOrCreate().sparkContext.stop()

spark = (SparkSession
             .builder
             .appName('test15-spark')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate()
)

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53904)
Traceback (most recent call last):
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/site-packages/pyspark/a

In [3]:
PATH = os.getcwd()
PATH_Data = PATH+'/data/Test1'
PATH_Result = PATH+'/data/results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

PATH:        /Users/gaeldelescluse/Documents/OpenClassRooms/2.Projets/Projet11/ai-cloud-computing-spark/train
PATH_Data:   /Users/gaeldelescluse/Documents/OpenClassRooms/2.Projets/Projet11/ai-cloud-computing-spark/train/data/Test1
PATH_Result: /Users/gaeldelescluse/Documents/OpenClassRooms/2.Projets/Projet11/ai-cloud-computing-spark/train/data/results


In [4]:
df = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)
df = df.withColumn('label', element_at(split(df['path'], '/'),-2))

In [5]:
df.show()

+--------------------+-------------------+------+--------------------+--------------+
|                path|   modificationTime|length|             content|         label|
+--------------------+-------------------+------+--------------------+--------------+
|file:/Users/gaeld...|2021-09-12 19:25:42|  5656|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5627|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5613|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5602|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5599|[FF D

In [25]:
def image_to_vector(image_binary):
    image_array = np.frombuffer(image_binary, np.uint8)
    # Lire l'image à l'aide d'OpenCV
    image = cv2.imdecode(image_array, cv2.IMREAD_GRAYSCALE)  # ou cv2.IMREAD_COLOR pour des images en couleur
    # Aplatir l'image en un vecteur 1D
    image_vector = image.flatten()
    # Normaliser les pixels (optionnel)
    image_vector = image_vector / 255.0
    return image_vector.tolist()

In [26]:
image_to_vector_udf = udf(image_to_vector, ArrayType(FloatType()))

In [27]:
df = df.withColumn('image_vector', image_to_vector_udf('content'))

In [28]:
def list_to_vector(image_list):
    return Vectors.dense(image_list)

list_to_vector_udf = udf(list_to_vector, VectorUDT())

In [29]:
df = df.withColumn('image_vector', list_to_vector_udf('image_vector'))

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['image_vector'],outputCol='features')
df_test = assembler.transform(df)

In [30]:
df.show()

+--------------------+-------------------+------+--------------------+--------------+--------------------+
|                path|   modificationTime|length|             content|         label|        image_vector|
+--------------------+-------------------+------+--------------------+--------------+--------------------+
|file:/Users/gaeld...|2021-09-12 19:25:42|  5656|[FF D8 FF E0 00 1...|Apple Braeburn|[1.0,1.0,1.0,1.0,...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5627|[FF D8 FF E0 00 1...|Apple Braeburn|[1.0,1.0,1.0,1.0,...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5613|[FF D8 FF E0 00 1...|Apple Braeburn|[1.0,1.0,1.0,1.0,...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|Apple Braeburn|[1.0,1.0,1.0,1.0,...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|Apple Braeburn|[1.0,1.0,1.0,1.0,...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|Apple Braeburn|[1.0,1.0,1.0,1.0,...|
|file:/Users/gaeld...|2021-09-12 19:2

In [31]:
df.printSchema()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)
 |-- image_vector: vector (nullable = true)



In [32]:
df.select('image_vector').first()

Row(image_vector=DenseVector([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9961, 1.0, 1.0, 0.9961, 0.9961, 1.0, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9804, 0.9961, 1.0, 0.9843, 1.0, 0.9961, 0.9843, 1.0, 1.0, 0.9765, 0.9961, 0.9882, 0.9961, 0.9961, 0.9882, 1.0, 0.9922, 0.9882, 1.0, 0.8275, 0.5098, 0.4784, 0.451, 0.4078, 0.4392, 0.451, 0.4824, 0.5176, 0.7294, 0.9922, 0.9882, 0.9922, 1.0, 0.9922, 0.9882, 1.0, 0.9922, 0.9882, 0.9843, 0.9922, 0.9961, 1.0, 0.9765, 1.0, 0.9922, 0.9882, 1.0, 0.9922, 1.0, 1.0, 1.0, 1.0, 0.9961, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9961, 0.9961, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9961, 1.0, 1.0, 0.9882, 1.0, 1.0, 0.9882, 1.0, 0.9765, 0.9882, 1.0, 0.9843, 0.9882, 1.0, 1.0, 0.9882, 0.9647, 0.5725, 0.3725, 0.2706, 0.298, 0.3098, 0.3098, 0.302, 0.2549, 0.2353, 0.2353, 0.2118, 0.2235, 0.2196,

In [33]:
from pyspark.ml.linalg import DenseVector, SparseVector

def dense_to_sparse(dense_vec):
    size = len(dense_vec)
    indices = []
    values = []
    
    for i, value in enumerate(dense_vec):
        if value != 1.0:  # Suppose que 1.0 est la valeur fréquente
            indices.append(i)
            values.append(value)
    
    return SparseVector(size, indices, values)

# UDF pour appliquer la conversion
dense_to_sparse_udf = udf(dense_to_sparse, VectorUDT())

# Appliquer la transformation
df_sparse = df.withColumn("sparse_vector", dense_to_sparse_udf(df["image_vector"]))

# Afficher le résultat
df_sparse.show()

+--------------------+-------------------+------+--------------------+--------------+--------------------+--------------------+
|                path|   modificationTime|length|             content|         label|        image_vector|       sparse_vector|
+--------------------+-------------------+------+--------------------+--------------+--------------------+--------------------+
|file:/Users/gaeld...|2021-09-12 19:25:42|  5656|[FF D8 FF E0 00 1...|Apple Braeburn|[1.0,1.0,1.0,1.0,...|(10000,[16,19,20,...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5627|[FF D8 FF E0 00 1...|Apple Braeburn|[1.0,1.0,1.0,1.0,...|(10000,[16,17,18,...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5613|[FF D8 FF E0 00 1...|Apple Braeburn|[1.0,1.0,1.0,1.0,...|(10000,[16,17,18,...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|Apple Braeburn|[1.0,1.0,1.0,1.0,...|(10000,[16,17,18,...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|Apple Braeburn|[1.0,1.0,1.0,1.0,..

In [34]:
df_sparse.select('sparse_vector').first()

Row(sparse_vector=SparseVector(10000, {16: 0.9961, 19: 0.9961, 20: 0.9961, 22: 0.9961, 23: 0.9961, 24: 0.9961, 25: 0.9961, 26: 0.9961, 27: 0.9804, 28: 0.9961, 30: 0.9843, 32: 0.9961, 33: 0.9843, 36: 0.9765, 37: 0.9961, 38: 0.9882, 39: 0.9961, 40: 0.9961, 41: 0.9882, 43: 0.9922, 44: 0.9882, 46: 0.8275, 47: 0.5098, 48: 0.4784, 49: 0.451, 50: 0.4078, 51: 0.4392, 52: 0.451, 53: 0.4824, 54: 0.5176, 55: 0.7294, 56: 0.9922, 57: 0.9882, 58: 0.9922, 60: 0.9922, 61: 0.9882, 63: 0.9922, 64: 0.9882, 65: 0.9843, 66: 0.9922, 67: 0.9961, 69: 0.9765, 71: 0.9922, 72: 0.9882, 74: 0.9922, 79: 0.9961, 94: 0.9961, 95: 0.9961, 123: 0.9961, 126: 0.9882, 129: 0.9882, 131: 0.9765, 132: 0.9882, 134: 0.9843, 135: 0.9882, 138: 0.9882, 139: 0.9647, 140: 0.5725, 141: 0.3725, 142: 0.2706, 143: 0.298, 144: 0.3098, 145: 0.3098, 146: 0.302, 147: 0.2549, 148: 0.2353, 149: 0.2353, 150: 0.2118, 151: 0.2235, 152: 0.2196, 153: 0.2157, 154: 0.2, 155: 0.2706, 156: 0.298, 157: 0.2588, 158: 0.2706, 159: 0.2275, 160: 0.2157, 161

In [35]:
pca = PCA(k=20, inputCol="sparse_vector", outputCol="pcaFeatures")
model = pca.fit(df_sparse)

24/05/28 23:21:14 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/28 23:22:05 ERROR Executor: Exception in task 0.0 in stage 11.0 (TID 44)4]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newArray(Native Method)
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2098)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2390)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2228)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1687)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:489)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInput

Py4JJavaError: An error occurred while calling o283.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 44) (192.168.0.15 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newArray(Native Method)
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2098)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2390)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2228)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1687)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:48

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [36]:
spark.stop()

ConnectionRefusedError: [Errno 61] Connection refused